In [1]:
import warnings
warnings.filterwarnings('ignore')

from src.data_processing import *
from src.grid_search import *
import os
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
import pickle

data_type = 'float32'

In [2]:
input_test = '../data/test_final_group_info.csv'
input_train = '../data/train_final_group_info.csv'

cell_type = 'All'

In [3]:
train_X, train_y, test_X, test_y, custom_cv = data_prep(input_test, input_train,
                                                        cell_type, binarization=False)

Finished data prepration for All


In [10]:
cpm_gene_file = '../AgingClock/Data/GSE65765_CPM.csv'
predictor_gene_file = '../AgingClock/Data/Predictor_Genes.csv'

In [11]:
predictor_gene_df = pd.read_csv(predictor_gene_file, index_col=0)

In [12]:
predictor_gene_df

,ElasticNet_Coef
WormBaseID,
WBGene00012747,-3.918385
WBGene00011554,-3.661531
WBGene00002259,-3.441976
WBGene00018196,-3.357286
WBGene00017813,-3.152588
...,...
WBGene00021601,3.507537
WBGene00022145,3.584022
WBGene00219950,3.599956


In [13]:
cpm_df = pd.read_csv(cpm_gene_file, index_col=0, sep='\t')

In [14]:
cpm_df

,SRR1793993,SRR1793991,SRR1793994,SRR1793992
WBGene00197333,0.000000,0.000000,0.000000,0.000000
WBGene00198386,0.000000,0.000000,0.000000,0.000000
WBGene00015153,3.780174,0.510354,4.718708,2.389905
WBGene00002061,169.240815,412.628597,274.733671,351.612558
WBGene00255704,1.907427,0.061861,1.234644,0.505892
...,...,...,...,...
WBGene00010965,234.526846,372.511713,253.220421,472.206803
WBGene00014471,0.017340,0.000000,0.033826,0.000000
WBGene00010966,54.483057,54.545971,19.483698,89.508039
WBGene00010967,78.117815,59.618577,86.492735,76.459508


In [15]:
cpm_df = cpm_df.T

In [16]:
df = cpm_df
df_bin = df.copy()
df_bin = df_bin.filter(regex='WBG')

In [17]:
df_bin

,WBGene00197333,WBGene00198386,WBGene00015153,WBGene00002061,WBGene00255704,WBGene00235314,WBGene00001177,WBGene00169236,WBGene00219784,WBGene00015152,...,WBGene00010964,WBGene00014467,WBGene00014468,WBGene00014469,WBGene00014470,WBGene00010965,WBGene00014471,WBGene00010966,WBGene00010967,WBGene00014473
SRR1793993,0.0,0.0,3.780174,169.240815,1.907427,0.277444,59.320986,0.0,0.000000,1.283178,...,858.949156,0.0,0.000000,0.0,0.052021,234.526846,0.017340,54.483057,78.117815,0.000000
SRR1793991,0.0,0.0,0.510354,412.628597,0.061861,0.061861,22.239044,0.0,0.015465,0.201048,...,1049.982885,0.0,0.015465,0.0,0.015465,372.511713,0.000000,54.545971,59.618577,0.000000
SRR1793994,0.0,0.0,4.718708,274.733671,1.234644,0.118391,42.400721,0.0,0.000000,0.642691,...,664.255412,0.0,0.101478,0.0,0.000000,253.220421,0.033826,19.483698,86.492735,0.016913
SRR1793992,0.0,0.0,2.389905,351.612558,0.505892,0.069778,20.497358,0.0,0.017445,1.308342,...,1298.799849,0.0,0.034889,0.0,0.000000,472.206803,0.000000,89.508039,76.459508,0.000000


In [18]:
 df_bin[df_bin == 0] = np.nan

In [21]:
df_bin['Median'] = df_bin.median(axis=1)
df_bin = df_bin.filter(regex='WBG').div(df_bin.Median, axis=0)

In [23]:
df_bin[df_bin.isna()] = 0
df_bin[df_bin <= 1] = 0
df_bin[df_bin > 1] = 1
df_bin

,WBGene00197333,WBGene00198386,WBGene00015153,WBGene00002061,WBGene00255704,WBGene00235314,WBGene00001177,WBGene00169236,WBGene00219784,WBGene00015152,...,WBGene00010964,WBGene00014467,WBGene00014468,WBGene00014469,WBGene00014470,WBGene00010965,WBGene00014471,WBGene00010966,WBGene00010967,WBGene00014473
SRR1793993,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
SRR1793991,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
SRR1793994,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
SRR1793992,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [24]:
binarized_df = df_bin

In [26]:
df_for_prediction = binarized_df[list(set(predictor_gene_df.index))]
df_for_prediction = df_for_prediction.T
df_for_prediction

,SRR1793993,SRR1793991,SRR1793994,SRR1793992
WBGene00019012,1.0,0.0,1.0,0.0
WBGene00012822,0.0,0.0,0.0,0.0
WBGene00012475,0.0,0.0,0.0,0.0
WBGene00016846,1.0,1.0,1.0,1.0
WBGene00023401,0.0,1.0,1.0,0.0
...,...,...,...,...
WBGene00008969,1.0,1.0,1.0,1.0
WBGene00021773,0.0,0.0,0.0,0.0
WBGene00000727,0.0,0.0,0.0,0.0
WBGene00008127,0.0,1.0,0.0,0.0


In [27]:
df_for_prediction = df_for_prediction.join(predictor_gene_df)
df_for_prediction

,SRR1793993,SRR1793991,SRR1793994,SRR1793992,ElasticNet_Coef
WBGene00019012,1.0,0.0,1.0,0.0,2.075180
WBGene00012822,0.0,0.0,0.0,0.0,2.337841
WBGene00012475,0.0,0.0,0.0,0.0,2.503940
WBGene00016846,1.0,1.0,1.0,1.0,-0.277555
WBGene00023401,0.0,1.0,1.0,0.0,0.444193
...,...,...,...,...,...
WBGene00008969,1.0,1.0,1.0,1.0,-1.011855
WBGene00021773,0.0,0.0,0.0,0.0,-1.769997
WBGene00000727,0.0,0.0,0.0,0.0,1.328427
WBGene00008127,0.0,1.0,0.0,0.0,-1.712661


In [29]:
prediction_result_dict = {}
prediction_result_dict['Sample'] = []
prediction_result_dict['Predicted_Biological_Age'] = []
    # loop over the samples (columns) in the DataFrame, predict the biological age in hours and save it to the
    # dictionary
intercept=103.54631743289005
for i in range(len(df_for_prediction.columns) - 1):
    prediction_result_dict['Sample'].append(df_for_prediction.columns[i])
    # Sum up all coefficients for genes that are 1 and thereby contribute to the biological age
    prediction_result_dict['Predicted_Biological_Age'].append(
        sum(df_for_prediction.iloc[:, i] * df_for_prediction.ElasticNet_Coef) + intercept)

prediction_result_df = pd.DataFrame(prediction_result_dict)
prediction_result_df = prediction_result_df.set_index('Sample')

In [ ]:
 df_for_prediction = preproc_new_data(cpm_gene_file, predictor_gene_file, sep=cpm_gene_file_sep, filter_genes=filter_genes)

    prediction_result_dict = {}
    prediction_result_dict['Sample'] = []
    prediction_result_dict['Predicted_Biological_Age'] = []
    # loop over the samples (columns) in the DataFrame, predict the biological age in hours and save it to the
    # dictionary
    for i in range(len(df_for_prediction.columns) - 1):
        prediction_result_dict['Sample'].append(df_for_prediction.columns[i])
        # Sum up all coefficients for genes that are 1 and thereby contribute to the biological age
        prediction_result_dict['Predicted_Biological_Age'].append(
            sum(df_for_prediction.iloc[:, i] * df_for_prediction.ElasticNet_Coef) + intercept)

    prediction_result_df = pd.DataFrame(prediction_result_dict)
    prediction_result_df = prediction_result_df.set_index('Sample')

In [30]:
prediction_result_df

,Predicted_Biological_Age
Sample,
SRR1793993,182.353657
SRR1793991,14.950266
SRR1793994,251.269911
SRR1793992,32.134522


In [33]:
import pandas as pd
import scipy.special
import numpy as np



def calculate_Bio_Age_correction(df, age_column='Bio_Age'):
    '''

    Calculate the second correction factor for every biological age of every sample (row) in df.


    :param df: Pandas DataFrame with a row for each Sample.
    Columns contain Genes, one column with the biological age, as well as possible further meta-data,i.e.
    the Strain, Treatment, RNAi, GEO accession number
    :return: A copy of df with the updated/corrected biological age
    '''
    df_corrected = df.copy()
    bio = df_corrected[age_column].values
    median = 372 # 15.5 days
    sd = (192 / 3) # 8/3 days

    # scipy.special.erf(((median - y) / sd) / np.sqrt(2)) calculates the number of SDs that the biological age y is
    # away from the median.
    # This value divided by 2 is the area that remains on the right side of the median.
    # We add 0.5 for the left side (50%) of the median to get the (approximated) total remaining area under the curve
    # To approximate at which percentage the new median of this truncated bell curve would be,
    # we divide the number by 2, in total:
    # (((scipy.special.erf(((median - y) / sd) / np.sqrt(2)) / 2) + 0.5) / 2)

    # If the biological age would be the median, i.e. y=372 hours, the right half of the bell curve would be trimmed.
    # This would leave just the left side of the curve, for which the new median can be approximated by the value
    # at which the area under the curve is 50%/2=25%.
    #
    # To calculate the biological age at which we get this percentage, we use the inverse error function
    # scipy.special.erfinv() to calculate how many SDs the new percentage value is apart from the original median at
    # 50%, i.e. scipy.special.erfinv(0.5 - X), where X is the approximated value explained above. This value needs to
    # be scaled to result in SDs by multiplying it with 2-times the square root of 2.
    # To calculate the new median, i.e. the rescaled biological age of the population,
    # we have to subtract the old median y by the number SDs we just calculated times 192/3,
    # which is the SD of the curve, i.e. y- numberOfSDs*sd

    corrected_bio_age = [(y - (scipy.special.erfinv(0.5 - (((scipy.special.erf(((median - y) / sd) /
                                                                  np.sqrt(2)) / 2) + 0.5) / 2)) * np.sqrt(2) * 2 * sd))
            for y in bio]
    df_corrected['Corrected_Biological_Age'] = corrected_bio_age

    return df_corrected


def test_correction():
    df = pd.read_csv('../AgingClock/Data/bio_age_example.csv')
    corrected_df = calculate_Bio_Age_correction(df)
    print(corrected_df)

In [34]:
test_correction()

          Run     Bio_Age  Corrected_Biological_Age
0  SRR1793991   26.571429                 26.571426
1  SRR1793992   26.571429                 26.571426
2  SRR1793993  212.571429                212.060642
3  SRR1793994  318.857143                302.516296


In [37]:
import numpy as np
import pandas as pd



def make_binary(df, filter_genes='WBG'):
    '''
    Takes a Pandas DataFrame with Count-Per-Million normalized read counts and returns a binarized version of it
    :param df: Pandas DataFrame with a row for each Sample.
    Columns contain Count-Per-Million(CPM) read counts, and possibly meta-data,i.e.
    the Strain, Treatment, RNAi, Biological Age, GEO accession number
    :param filter_genes: Filter columns names by filter_genes to compute the binarization only on the genes.
    :return: A binarized copy of the original data without meta-information
    '''
    df_bin = df.copy()
    df_bin = df_bin.filter(regex=filter_genes)  # get the gene columns
    df_bin[df_bin == 0] = np.nan  # mask 0-genes that skew the median
    df_bin['Median'] = df_bin.median(axis=1)  # calculate the median for each row and append a column
    # divide each value, i.e. CPM, by the sample-median
    df_bin = df_bin.filter(regex=filter_genes).div(df_bin.Median, axis=0)
    # set values smaller than the median to 0 and 1 otherwise
    df_bin[df_bin.isna()] = 0
    df_bin[df_bin <= 1] = 0
    df_bin[df_bin > 1] = 1

    return df_bin





def preproc_new_data(cpm_gene_file, predictor_gene_file, sep='\t', filter_genes='WBG'):
    '''
    Read a CPM containing CSV-file, binarize it and return only the subset of genes relevant for the prediction

    :param cpm_gene_file: A file with non-duplicate WormbaseID gene identifiers in the first column and 1 or more columns with
    gene expression samples.
    For example:

    Gene_ID	SRR6207028	SRR6207027	SRR6207030	SRR6207029
    WBGene00007063	2.9784207155	1.4274445525	0.9728337164	3.2820503369
    WBGene00200402	0	0	0	0
    WBGene00007064	21.8278717092	16.4387110019	32.8859628004	30.5373379176
    WBGene00044951	0	0	0	0
    WBGene00195865	0	0	0	0
    WBGene00197051	0	0	0	0
    WBGene00199694	0	0	0	0
    WBGene00199940	0	0	0	0
    WBGene00201913	0	0	0	0
    WBGene00007065	4.6394630376	2.9846567916	3.0157845209	5.1371222665
    ....

    :param predictor_gene_file:  The file with the elastic net coefficients for the 576 genes (Predictor_Genes.csv):
    WormBaseID	ElasticNet_Coef
    WBGene00012747	-3.9183849126
    WBGene00011554	-3.6615313969
    WBGene00002259	-3.4419756727
    WBGene00018196	-3.3572860482
    ....

    :param sep: Delimiter to use, default '\t'
    :param filter_genes: Start of the gene names in the columns. Default for C. elegans: 'WBG'
    :return: A DataFrame with binarized CPMs for the 576 genes relevant for the prediction and an additional last column with
    the elastic net coefficients
    '''

    # Read the predictor_gene_file
    predictor_gene_df = pd.read_csv(predictor_gene_file, index_col=0)

    # Read the new data and transpose it to make it fit for binarization
    cpm_df = pd.read_csv(cpm_gene_file, index_col=0, sep=sep)
    cpm_df = cpm_df.T
    binarized_df = make_binary(cpm_df, filter_genes=filter_genes)

    # take the subset of genes that is relevant for the prediction and transpose the DataFrame to join the elastic
    # net coefficients
    df_for_prediction = binarized_df[list(set(predictor_gene_df.index))]
    df_for_prediction = df_for_prediction.T
    # attach the elastic net coefficients to the DataFrame
    df_for_prediction = df_for_prediction.join(predictor_gene_df)

    return df_for_prediction


def predict(cpm_gene_file, predictor_gene_file, cpm_gene_file_sep='\t', intercept = 103.54631743289005, filter_genes='WBG'):
    '''

    :param cpm_gene_file: A file with non-duplicate WormbaseID gene identifiers in the first column and 1 or more columns with
    gene expression samples.
    For example:
    Gene_ID	SRR6207028	SRR6207027	SRR6207030	SRR6207029
    WBGene00007063	2.9784207155	1.4274445525	0.9728337164	3.2820503369
    WBGene00200402	0	0	0	0
    WBGene00007064	21.8278717092	16.4387110019	32.8859628004	30.5373379176
    WBGene00044951	0	0	0	0
    WBGene00195865	0	0	0	0
    WBGene00197051	0	0	0	0
    WBGene00199694	0	0	0	0
    WBGene00199940	0	0	0	0
    WBGene00201913	0	0	0	0
    WBGene00007065	4.6394630376	2.9846567916	3.0157845209	5.1371222665
    ....

    :param predictor_gene_file:  The file with the elastic net coefficients for the 576 genes (Predictor_Genes.csv):
    WormBaseID	ElasticNet_Coef
    WBGene00012747	-3.9183849126
    WBGene00011554	-3.6615313969
    WBGene00002259	-3.4419756727
    WBGene00018196	-3.3572860482
    ....

    :param cpm_gene_file_sep: Delimiter to use, default '\t'
    :param intercept: The intercept has to be added to get the final prediction, default is the intercept for C. elegans. The intercept for human fibroblasts is 59.626.
    :param filter_genes: Start of the gene names in the columns. Default for C. elegans: 'WBG'
    :return: Pandas DataFrame with the sample names (column names) of cpm_gene_file as the index and
    the predicted biological age in hours in the second column
    '''

    
    

    # Read a CPM containing CSV-file (cpm_gene_file), binarize the gene expression and
    # get only the subset of genes relevant for the prediction (predictor_gene_file)
    # the last column of df_for_prediction contains the elastic net coefficients for the prediction
    df_for_prediction = preproc_new_data(cpm_gene_file, predictor_gene_file, sep=cpm_gene_file_sep, filter_genes=filter_genes)

    prediction_result_dict = {}
    prediction_result_dict['Sample'] = []
    prediction_result_dict['Predicted_Biological_Age'] = []
    # loop over the samples (columns) in the DataFrame, predict the biological age in hours and save it to the
    # dictionary
    for i in range(len(df_for_prediction.columns) - 1):
        prediction_result_dict['Sample'].append(df_for_prediction.columns[i])
        # Sum up all coefficients for genes that are 1 and thereby contribute to the biological age
        prediction_result_dict['Predicted_Biological_Age'].append(
            sum(df_for_prediction.iloc[:, i] * df_for_prediction.ElasticNet_Coef) + intercept)

    prediction_result_df = pd.DataFrame(prediction_result_dict)
    prediction_result_df = prediction_result_df.set_index('Sample')

    return prediction_result_df

def test_prediction():
    '''
    Usage Example.
    The CPM gene file and the predictor gene file can be found in data/
    :return: 
    '''
    cpm_gene_file = '../AgingClock/Data/GSE65765_CPM.csv'
    predictor_gene_file = '../AgingClock/Data/Predictor_Genes.csv'
    res = predict(cpm_gene_file, predictor_gene_file, cpm_gene_file_sep='\t')
    print(res)


In [39]:
test_prediction()

            Predicted_Biological_Age
Sample                              
SRR1793993                182.353657
SRR1793991                 14.950266
SRR1793994                251.269911
SRR1793992                 32.134522


In [ ]:
def make_binary(df):
    df_bin = df.copy()
    df_bin[df_bin == 0] = np.nan  # mask 0-genes that skew the median
    df_bin['Median'] = df_bin.median(axis=1)  # calculate the median for each row and append a column
    # divide each value, i.e. CPM, by the sample-median
    df_bin = df_bin.filter(regex=filter_genes).div(df_bin.Median, axis=0)
    # set values smaller than the median to 0 and 1 otherwise
    df_bin[df_bin.isna()] = 0
    df_bin[df_bin <= 1] = 0
    df_bin[df_bin > 1] = 1

    return df_bin
